# Tutorial 5: RJMCMC with Eryn

张铭昊 北京化工大学 zmh780674484@163.com

In [ ]:
教程5：基于Eryn的随机跳跃马尔可夫链蒙特卡洛（RJMCMC）方法

In the fifth tutorial, we will look at running Eryn when the underlying model is changing. This means we will be using Reversible-Jump MCMC, also referred to as trans-dimensional MCMC. In this tutorial, there will only be 2 tasks because RJMCMC is complicated and putting together an RJMCMC run takes time and effort. 

  第五个教程将介绍在基础模型发生变化时运行Eryn的情况。这意味着我们将使用可逆跳跃马尔可夫链蒙特卡洛（Reversible-Jump MCMC），也叫做跨维度MCMC。在这个教程中，只有两个任务，因为RJMCMC非常复杂，并且组织一个RJMCMC运行需要时间和精力。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from lisatools.utils.constants import *
from copy import deepcopy  # can be useful

## Task 1: How many Gaussian pulses?

In [ ]:
任务1：有多少个高斯脉冲？

The first RJ problem we will look at is determining how many 2D Gaussian pulses exist in a noisy set of data. This is an example you can find in the more advanced Eryn tutorials. To keep this simple in the time we have alotted, we will use a proposal based on the prior distribution (this is the default). If you set `rj_moves=True` in the setup of `EnsembleSampler`, it will automatically generate RJ proposals from the prior. However, you must be careful when using multiple branches (model types) because this proposal, by default, always proposes to change each model in the sampler by 1 model count, either add or remove. 

Useful documentation:
* [EnsembleSampler](https://mikekatz04.github.io/Eryn/html/user/ensemble.html#eryn.ensemble.EnsembleSampler)
* [State](https://mikekatz04.github.io/Eryn/html/user/state.html#eryn.state.State)
* [uniform_dist](https://mikekatz04.github.io/Eryn/html/user/prior.html#eryn.prior.uniform_dist)
* [ProbDistContainer](https://mikekatz04.github.io/Eryn/html/user/prior.html#eryn.prior.ProbDistContainer)
* [GaussianMove](https://mikekatz04.github.io/Eryn/html/user/moves.html#eryn.moves.GaussianMove)
* [DistributionGenerateRJ](https://mikekatz04.github.io/Eryn/html/user/moves.html#eryn.moves.DistributionGenerateRJ)

  要解决的第一个随机跳跃（RJ）问题，即确定在一组带有噪声的数据中存在多少个二维高斯脉冲。这是在更高级的Eryn教程中可以找到的一个示例。为了在我们规定的时间内保持简单，我们将使用基于先验分布的建议（是默认设置）。若EnsembleSampler的设置中将rj_moves设为True，它将自动生成来自先验分布的RJ建议。但是，当使用多个分支（模型类型）时，你必须小心使用此建议，因为默认情况下，该建议总是建议通过增加或减少1个模型计数来改变采样器中的每个模型。

In [ ]:
from eryn.ensemble import EnsembleSampler
from eryn.state import State
from eryn.prior import ProbDistContainer, uniform_dist
from eryn.moves import GaussianMove
from eryn.backends import HDFBackend
from chainconsumer import ChainConsumer, Chain

We will do the initial setup for you. Here we are going to set the x and y dimensions of our 2D grid.

  我们为你进行了最初的设置。在这里，我们将于2D图格上设置x,y两个维度

In [ ]:
num     = 100 # the number of step for each dimension
lowlim  = -10 # Low limit on each axis
highlim = 10  # high limit on each axis
npulses = 10  # The # of injected pulses

dx = (highlim - lowlim) / num # Get the discritization

x, y = np.mgrid[lowlim:highlim:dx, lowlim:highlim:dx]  # Generate the grid

Now, we will draw initial parameters for each set of pules. Each pulse gets an amplitude, x, and y value.

  现在，我们将调整每一个脉冲的初始参数。每一个脉冲会被赋予一个振幅，x值与y值

In [ ]:
Amp    = np.random.uniform(.5, 2.0, size=(npulses)) # Draw an amplitude
spread = .2  # Keep their spread as fixed for simplicity.
sigma  = spread * np.diag(np.ones(2))

edges = 2 # Utility parameter, in order to avoid having signals at the border of our data set

# Draw the coordinates parameters
# generates random x and y coordinates
inj_coordinates = np.random.uniform(lowlim+edges, highlim-edges, size=(npulses, 2))

# Gather all parameters here
gauss_inj_params = np.concatenate([Amp[:, None], inj_coordinates], axis=1)

print(' * Parameters injected: \n\n', np.matrix(gauss_inj_params))

We have also filled in the Gaussian function and Likelihood to avoid any issues. The next few cells give an idea of the injection data. 

  为了避免一些问题，我们同时也添加了高斯函数与似然值函数。接下来的文本将给你提供关于注入数据的一个思路

In [ ]:
# First we compute some constant terms of the Gaussian models (reminder: we have assumed a fixed spread for each pulse)
sigma_det = np.linalg.det(sigma)
sigma_inv = np.linalg.inv(sigma)
norm      = np.sqrt((2*np.pi)**2 * sigma_det)

def gaussian(X, Y, a, b, c):
    x = X[:,0]
    y = Y[0,:]
    # breakpoint()
    A = np.exp(-((x[None, :] - b[:, None]) ** 2) / (2 * sigma[0,0]))
    B = np.exp(-((y[None, :] - c[:, None]) ** 2) / (2 * sigma[1,1]))
    # breakpoint()
    C =  A[:, None, :] * B[:, :, None] # (np.expand_dims(A,axis=0) * np.expand_dims(np.transpose(B),axis=2))

    return a[:, None, None] * C / norm

def log_prob_fn(x1, X, Y, data, sig):

    a = x1[:, 0]
    b = x1[:, 1]
    c = x1[:, 2]
    n = num * num

    template  = np.sum(gaussian(X, Y, a, b, c), axis=0)

    llh = - 0.5 * ( np.sum(((template - data)) ** 2) )
    llh *= 1/sig**2
    llh += - n*np.log(sig) - .5 * n * np.log(2.*np.pi)
    return llh

In [ ]:
noise = spread * np.random.randn( num, num ) # Draw the random points for the noise

# Generate the data-set
injection = np.zeros( (num, num) )
injection = np.sum(gaussian(x, y, gauss_inj_params[:, 0], gauss_inj_params[:, 1], gauss_inj_params[:, 2]), axis=0)
data = injection + noise


In [ ]:
from matplotlib import cm

plt.figure(figsize=(17,7))
plt.subplot(121)
cf = plt.contourf(y, x, injection, 10, cmap=cm.PuBu)
plt.scatter(gauss_inj_params[:,1], gauss_inj_params[:,2], marker='x', color='#DC143C')
plt.colorbar(cf)
plt.xlabel(r'$x$')
plt.ylabel(r'$y$')
plt.subplot(122)
cf = plt.contourf(y, x, data, 10, cmap=cm.PuBu)
plt.scatter(gauss_inj_params[:,1], gauss_inj_params[:,2], marker='x', color='#DC143C')
plt.xlabel(r'$x$')
plt.ylabel(r'$y$')
plt.colorbar(cf)
plt.show()


Now that we have most of the initial setup. We now need to build our sampling run. Let's start by choosing settings and building base requirements: `ndims`, `nleaves_max`, `nleaves_min`, `branch_names`, `ntemps`, `nwalkers`.

  既然我们完成了大多数初始设定，我们现在需要建立并运行采样器。让我们在构建采样运行之前，选择设置并构建基本要求，包括ndims、nleaves_max、nleaves_min、branch_names、ntemps和nwalkers。

In [ ]:
ntemps = 10
nwalkers = 30

ndims        = {"pulse": 3}           
nleaves_max  = {"pulse": 2*npulses} 
nleaves_min = {"pulse": 0}
branch_names = ["pulse"]

Now we will setup our prior function. Make sure you initialize your prior with the `ProbDistContainer` object from Eryn. 
* For the amplitude, a uniform prior from 0.5 to 2.0.
* For x and y, uniform priors across the acceptable ranges.

  现在我们将设定我们的先验分布函数。请确保使用在Eryn中的ProbDistContainer对象初始化了你的先验分布

In [ ]:
priors = {
    "pulse": ProbDistContainer({
        0: uniform_dist(0.5, 2.),
        1: uniform_dist(lowlim, highlim),
        2: uniform_dist(lowlim, highlim),
    })
}

When using RJMCMC, we must input a value for `moves` in the `EnsembleSampler`. This is because the default proposal (Stretch proposal) does not work in varying dimensionality. We will keep this simple and use a Gaussian proposal centered on the current point. We recommend a diagonal covariance matrix with the same covariance for each parameter. There are many ways to do this. 

Useful Documentation:
* [GaussianMove](https://mikekatz04.github.io/Eryn/html/user/moves.html#eryn.moves.GaussianMove)

  在使用RJMCMC时，我们必须为EnsembleSampler的moves参数输入一个值。这是因为默认的建议方法（Stretch建议）在维度变化的情况下不起作用。为了保持简单，我们将使用以当前点为中心的高斯建议。我们建议使用对角协方差矩阵，并且每个参数具有相同的协方差。这有多种实现方法。

In [ ]:
# imports 
from eryn.moves import GaussianMove

In [ ]:
factor = 0.01
cov    = {"pulse": np.diag(np.ones(3)) * factor}
moves = GaussianMove(cov)

Now we will instantiate the `EnsembleSampler`. For the tempering, set `tempering_kwargs=dict(ntemps=ntemps)`. 

  在这一步中，我们将实例化EnsembleSampler。对于温度调节（tempering），请设置tempering_kwargs=dict(ntemps=ntemps)。

In [ ]:
ensemble = EnsembleSampler(
    nwalkers,
    ndims,  
    log_prob_fn,
    priors,
    args=[x, y, data, sigma_det],
    tempering_kwargs=dict(ntemps=ntemps),
    nbranches=len(branch_names),
    branch_names=branch_names,
    nleaves_max=nleaves_max,
    nleaves_min=nleaves_min,
    moves=moves,
    rj_moves=True,
)

With the sampler prepared, we just need to generate our starting points. There are also many choices for this. In RJMCMC, it is more complicated because of the model uncertainty. For this simplified example, we are going to start by drawing **1 Gaussian per walker** from the prior. It will add more as the goes. Your goal here is to produce a `coords` dictionary of shape `(ntemps, nwalkers, nleaves_max, ndim)`. You can sample every source in that `coords` array because we will direct to have **only one** leaf per walker with the `inds` dictionary. `inds` should be a boolean array of shape `(ntemps, nwalkers, nleaves_max)`. You then index this and set one value to True per walker. Fill these dictionaries into a `State` object.

  准备好采样器之后，我们只需要生成起始点。对于起始点来说，也有很多选择。在RJMCMC中，由于模型的不确定性，这变得更加复杂。对于这个简化的示例，我们将从先验分布中为每个游走单元绘制1个高斯分布。随着采样的进行，会逐渐添加更多的高斯分布。你的目标是生成一个形状为(ntemps, nwalkers, nleaves_max, ndim)的coords字典。你可以在该coords数组中对每个源进行采样，因为我们将指定每个游走单元只有一个叶子节点，并使用inds字典。inds应该是一个形状为(ntemps, nwalkers, nleaves_max)的布尔数组。然后，你可以索引它并为每个walker设置一个True值。将这些字典填充到一个State对象中。

In [ ]:
coords = {
    name: priors[name].rvs(size=(ntemps, nwalkers, nleaves_max[name]))
    for name in branch_names
} 

inds_tmp = np.zeros(coords['pulse'].shape[:-1], dtype=bool)
inds_tmp[:, :, 0] = True

inds = {'pulse': inds_tmp}
state = State(coords, inds=inds)

Now run the sampler. We recommend a good burn in.

现在运行采样器，我们推荐你进行一个较为完善的烧入阶段

In [ ]:
nsteps = 5000
output = ensemble.run_mcmc(state, nsteps, burn=1000, progress=True, thin_by=1)

Now read out the number of leaves per walker in the cold chain using the `ensemble.backend`. You only need the `inds` array for this. Plot a histogram of number of pulses found in the cold chain. This is a posterior distribution on the model count. 

Useful Documentation:
* [Backend](https://mikekatz04.github.io/Eryn/html/user/backend.html#eryn.backends.Backend)

  要读取冷链中每个walker的叶子节点数量，可以使用ensemble.backend读取inds数组。这里只需要inds数组。绘制叶子节点数量的直方图，该直方图表示了模型数量的后验分布。

In [ ]:
nleaves = ensemble.backend.get_nleaves()["pulse"][:, 0].flatten() 

bins = (np.arange(1, nleaves_max['pulse'] + 2) - 0.5)  # Get maximum allowed number of leaves for the given branch

plt.hist(
    nleaves,
    bins=bins,
    color='#6495ed',
    alpha=0.9,
    lw=2,
    histtype='step',
    density=True,
    hatch='///'
)

# injection line 

plt.axvline(x=int(npulses), linestyle='--', lw=2, color='#DC143C')
plt.xticks(fontsize=12)
plt.xlim(1, 20)
plt.xlabel("$\#$ of peaks in the data")

### Question:
Is this what you expected? Does RJMCMC tell us what the "right" answer is? How do you think the noise effects this posterior plot? If we increase the noise, what type of changes do we expect and why?

Now we will look at the parameters of the recovered pulses to see how well we located the injected pulses. To do this, we are going to flatten all of samples from all leaf counts found, i.e. we will take every single source in the cold chain and plot all of these overlayed. **Hint**: use the `inds` array to your advantage here.

We recommend using `pandas` and `chainconsumer` to do this, but you can do it anyway you like. 

问题
  在这种情况下，这是你所希望的结果吗？RJMCMC能告诉我们"正确"的答案是什么吗？你怎么看待后验分布体现的噪声影响？如果增加噪声水平，我们期待会发生什么变化，为什么？
  现在，我们将查看恢复的脉冲的参数，以了解我们定位注入脉冲的效果。为了做到这一点，我们将展开所有找到的叶子节点数的样本，即将冷链中的每个单独的源都绘制在一起。提示：在这里可以利用inds数组。
  我们建议使用pandas和chainconsumer来完成这个任务，但你也可以选择其他方法。



In [ ]:
import pandas as pd

In [ ]:
pulse_parameters = ["$A$", r"$x$", r"$y$"]

samples_pulses = ensemble.backend.get_chain()['pulse'][ensemble.backend.get_inds()["pulse"]]

df = pd.DataFrame({key: val for key, val in zip(pulse_parameters, samples_pulses.T)})
c = ChainConsumer()
check = int(3)
c.add_chain(Chain(samples=df, name="Test", smooth=check))
fig = c.plotter.plot()

### Question:
How did we do? Was it a success? Could there be improvements? 

问题：
  我们做了什么？这件事算是成功吗？可有什么改进的地方吗？

## Task 2: Model selection with RJMCMC

In this task, you will use RJMCMC to do a direct model comparison. This will be the same model comparison used in Tutorial 3 comparing a Gaussian pulse with a Cauchy pulse. In tutorial 3, we used fixed-dimensional MCMC and thermodynamic integration to estimate the evidence of each model. Then we compared them to get the Bayes Factor. Using RJMCMC will produce a posterior distribution on the model selection index rather than a direct evidence for each model. The fraction of walkers that have highlighted one model over the other represents a proxy for the odds ratio. This is not strictly true when one model is much more favored than another. However, when model favorability is close, this direct computation is okay. 

For simplicity, we will consider the spread of the signal to be 1. Therefore, we are only fitting and comparing the amplitude and mean of the pulse. 

Once again, we will start by providing the functions for the Gaussian pulse, Cauchy pulse, and the Likelihood function. 

任务2：RJMCMC的模型选取
  在这个任务中，你将使用RJMCMC进行直接的模型比较。这与教程3中比较高斯脉冲和柯西脉冲的模型比较相同。在教程3中，我们使用固定维度的MCMC和热力学积分来估计每个模型的证据。然后我们将它们进行比较，得到贝叶斯因子。使用RJMCMC将产生一个关于模型选择指数的后验分布，而不是每个模型的直接证据。突出显示一种模型的walker的比例代表了一个比率。当一个模型比另一个模型更受青睐时，这种说法并不严格成立。然而，当模型的可取性接近时，这种直接计算是可行的。
  为了简单起见，我们将认为信号的传播范围为1。因此，我们只拟合和比较脉冲的振幅和均值。
  我们将再次首先从提供高斯脉冲、柯西脉冲和似然函数的函数开始。

In [ ]:
from scipy.stats import cauchy
def gaussian_pulse(x, a, b):
    f_x = a * np.exp(-((x - b) ** 2) / (2 * 1.0 ** 2))
    return f_x

def cauchy_pulse(x, a, b):
    f_x = a * cauchy.pdf(x - b)
    return f_x

def log_like_fn(params, t, data, sigma, which_template):

    pulse_gen = gaussian_pulse if which_template == "gauss" else cauchy_pulse
    template = pulse_gen(t, *params)

    ll = -0.5 * np.sum(((template - data) / sigma) ** 2, axis=-1)
    return ll

def log_like_wrap(params, *args):
    # must be careful about how these models are read into the Likelihood function
    # In this case, we will have one model with a single set of parameters 
    # and one model that is None
    
    assert len(params) == 2
    if params[0] is not None:
        assert params[1] is None
        # gauss will be first set of params
        args += ("gauss",)
        ll = log_like_fn(params[0][0], *args)

    else:
        assert params[0] is None
        # gauss will be first set of params
        args += ("cauchy",)
        ll = log_like_fn(params[1][0], *args)

    return ll

We will inject the Gausian pulse and compare to the Cauchy pulse.

  我们将注入高斯脉冲并用其与柯西脉冲进行比较。

In [ ]:
t_vals = np.linspace(-10.0, 10.0, 1000)
sigma = 3.5
amp_true = 4.0
mean_true = 0.0
true_data = gaussian_pulse(t_vals, amp_true, mean_true)
data = true_data + np.random.randn(*t_vals.shape) * sigma
cauchy_data = cauchy_pulse(t_vals, amp_true * 3, mean_true)
plt.plot(t_vals, data, label="data")
plt.plot(t_vals, true_data, label="gauss")
plt.plot(t_vals, cauchy_data, label="cauchy")
plt.legend()
# plt.plot(x_vals, np.exp(log_like_fn()))
# plt.plot(x_vals, np.exp(log_like_gauss(x_vals)))

We will follow the same setup order as the last problem. Start with the basic declarations (`ntemps`, `ndims`, etc.). For this problem, the max leaves for both models should be 1 and the min leaves for both models should be 0. 

  在这个问题中，我们将按照与上一个问题相同的设置顺序进行。首先进行基本的声明（ntemps、ndims等）。对于这个问题，两个模型的最大叶子节点数应该都是1，最小叶子节点数都是0。

In [ ]:
ntemps = 10
nwalkers = 30

ndims        = {"gauss": 2, "cauchy": 2}             
nleaves_max  = {"gauss": 1, "cauchy": 1}   
nleaves_min = {"gauss": 0, "cauchy": 0}
branch_names = ["gauss", "cauchy"]

Now we will put together the priors for both models. The priors are identical really for the two models. The amplitude prior should span the injection values. The mean prior should span the domain of time.

  现在我们将为两个模型组合先验分布。实际上，两个模型的先验分布是相同的。振幅的先验分布应该覆盖注入值的范围。均值的先验分布应该覆盖时域的范围。



In [ ]:
priors = {}
priors["gauss"] = ProbDistContainer({
    0: uniform_dist(0.0, 40.0),
    1: uniform_dist(t_vals.min(), t_vals.max())
}) 

priors["cauchy"] = ProbDistContainer({
    0: uniform_dist(0.0, 40.0),
    1: uniform_dist(t_vals.min(), t_vals.max())
}) 

Now we will produce the same type of `GaussianMove` that we built earlier. We need to make sure that to provide a covariance for each model! So, it must be a dictionary with entries for both models. The entries can really be the same though. 

  现在我们将创建与之前相同类型的GaussianMove。我们需要确保为每个模型提供一个协方差矩阵！因此，它必须是一个包含两个模型条目的字典。尽管条目可以相同。

In [ ]:
factor = 0.01
cov    = {"gauss": np.diag(np.ones(2)) * factor, "cauchy": np.diag(np.ones(2)) * factor}
moves = GaussianMove(cov)

Instantiate the `EnsembleSampler`.

 初始化EnsembleSampler

In [ ]:
sampler = EnsembleSampler(
    nwalkers,
    ndims,
    log_like_wrap,
    priors,
    nbranches=2,
    branch_names=branch_names,  
    tempering_kwargs=dict(ntemps=ntemps, Tmax=np.inf),
    args=(t_vals, data, sigma),
    rj_moves=True,
    moves=moves,
    nleaves_max=nleaves_max,
    nleaves_min=nleaves_min
)

Generate start points. **Note**: this is very important. This test will only work if:
* we have **2** models,
* each model has max leaves of 1 and min leaves of 0,
* and the starting points must all have **either** the Gaussian or Cauchy pulse, not both and not neither.

In this setup, each proposal will propose to switch the model and draw from its prior. 

Generate your start point and use the `inds` array to tell the sampler that each walker has only one model instance. 

In [ ]:
  生成起始点。注意：这非常重要。只有在以下条件下，这个测试才能成功进行：
  我们有两个模型，
  每个模型的最大叶子节点数为1，最小叶子节点数为0，
  起始点必须都是高斯脉冲或柯西脉冲，不能既是高斯脉冲也是柯西脉冲，也不能既不是高斯脉冲也不是柯西脉冲。

  在这个设置中，每个提议都会提出切换模型并从其先验分布中抽取样本。

  生成你的的起始点，并使用inds数组告诉采样器每个游走单元只有一个模型实例。

In [ ]:
start_point = {name: priors[name].rvs(size=(ntemps, nwalkers, 1)) for name in branch_names}


model_index = np.random.randint(0, 2, size=(ntemps, nwalkers, 1))

inds = {name: np.zeros((ntemps, nwalkers, 1), dtype=bool) for name in branch_names}

inds["gauss"][model_index == 0] = True
inds["cauchy"][model_index == 1] = True

assert np.all(inds["gauss"].astype(int) + inds["cauchy"].astype(int) == 1)
start_state = State(start_point, inds=inds)



Run the sampler with a burn-in.

  在经历烧入阶段的前提下运行采样器

In [ ]:
nteps = 500
sampler.run_mcmc(start_state, nsteps, progress=True, burn=100)

To find our posterior odds ratio, we need to get the fraction of cold-chain samples that exist in one of the two states. Calculate the odds ratio.

  为了计算我们的后验比率（posterior odds ratio），我们需要获得冷链样本的比例，那存在于两个状态中。计算后验比率。

In [ ]:

nleaves["gauss"][:, 0].sum() / np.prod(nleaves["gauss"][:, 0].shape)

### Question:
If we change the noise, what effect will this have on our results? What happens when one model is heavily favored so that the fraction is 1? Is this a proper estimate of the odds ratio? What could we do in this case where the initial odds ratio is 1 to actually estimate the odds ratio?

问题
  如果我们改变噪声，这会对我们的结果产生什么影响？当一个模型被严重偏好，使得比例为1时会发生什么？这是否是一个正确的后验比率估计？在初始后验比率为1的情况下，我们可以采取什么措施来真正估计后验比率？

总结与收获
  在这一章学习了如何在基础模型发生变化时使用RJMCMC方法，并使用Eryn算法来进行跳跃和模型选择，使用了RJ建议来确定在带有噪声的数据中存在多少个二维高斯脉冲，并向之前那样对模型进行了比较。
  RJMCMC方法也是一个需要大量时间系统性学习研究的内容，像本章这样的内容至少可以给我提供一个接触RJMCMC方法的窗口，让我在未来（有可能）进一步学习天文数据处理时能有一个参考。